In [ ]:
#import files
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from keras.models import load_model

In [ ]:
data = pd.read_csv('../input/diabetic-retinopathy-2015-data-colored-resized/trainLabels.csv')
data.head(5)

In [ ]:
# 0 - No DR
# 1 - Mild
# 2 - Moderate
# 3 - Severe
# 4 - Proliferative DR
class_to_folder = ['../input/diabetic-retinopathy-2015-data-colored-resized/colored_images/colored_images/No_DR/',
                   '../input/diabetic-retinopathy-2015-data-colored-resized/colored_images/colored_images/Mild/',
                   '../input/diabetic-retinopathy-2015-data-colored-resized/colored_images/colored_images/Moderate/',
                   '../input/diabetic-retinopathy-2015-data-colored-resized/colored_images/colored_images/Severe/',
                   '../input/diabetic-retinopathy-2015-data-colored-resized/colored_images/colored_images/Proliferate_DR/']

                   

In [ ]:
for ind in data.index:
  cls = data['level'][ind]
  path = class_to_folder[cls]+data['image'][ind]+".png"
  # print(cls, path)
  if cls in [1,2,3,4]:
    cls = "DR"
  else:
    cls = "No_DR"
  data['level'][ind] = cls
  data['image'][ind] = path

In [ ]:
data.info

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2, rescale=1./255, shear_range = 0.25, rotation_range= 25)

In [ ]:
train_generator = datagen.flow_from_dataframe(
    data, 
    x_col = 'image',
    y_col = 'level',
    subset='training',
    target_size=(224,224),
    shuffle = True,
)

val_generator = datagen.flow_from_dataframe(
    data, 
    x_col = 'image',
    y_col = 'level',
    subset='validation',
    target_size=(224,224)
)

In [ ]:
#Transfer Learning for Extraction
pre_trained = tf.keras.applications.InceptionResNetV2(
    include_top=False, 
    weights='imagenet',
    input_shape = (224,224,3)
)

In [ ]:
pre_trained.trainable = True

In [ ]:
#Neural Network For Classification
model = tf.keras.models.Sequential()
model.add(pre_trained)
model.add(tf.keras.layers.GlobalAveragePooling2D())
# ===================================================
# model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(1024, activation='relu'))
# model.add(tf.keras.layers.Dropout(0.2))
# ---------------------------------------------------
#Output layer
model.add(tf.keras.layers.Dense(2, activation = 'softmax'))
model.summary()

In [ ]:
#Compiling the Model
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics = ['accuracy'])


In [ ]:
#es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

In [ ]:
#Fit
history = model.fit(
          train_generator,
          validation_data = val_generator,
          epochs = 3,
          shuffle  = True)
          #callbacks = [es]


In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

In [ ]:
history = model.fit(
          train_generator,
          validation_data = val_generator,
          epochs = 2,
          shuffle  = True)

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

In [ ]:
model.save('model_01.h5')
#model = model.load('model_01.h5')